### Detecting insertion attacks by heuristics

According to Frontrunner-Jones insertion attacks can be detected the following way: 
 
A transfer event is triggered on the blockchain, whenever an ERC-20 token is traded.  
An Event combines the following transactional information E = (s,r,a,c,h,i,g):
- s: sender of tokens
- r: receiver of tokens
- a: number of transferred tokens
- c: contract address of token
- h: transaction hash
- i: transaction index
- g: gas price of transaction

Iterating block by block through all transfer events and checking if there are 3 events EA1, EV, EA2 for which the folloing 6 heuristics hold:  


#### Heuristic 1

Heuristic 1: 
- sender of EA1 must be identical to sender of EV and receiver of EA2
    - sA1 = sV = rA2 
- receiver of EA1 must be identical to sender of EA2.
    - rA1 = sA2 

#### Heuristics 2
- number of tokens bought by EA1 must be similar to the umber of tokens sold by EA2 ( difference of max 1%).

#### Heuristics 3
- token contract address of EA1, EV and EA2 must be identical
    - cA1 = cV2 = cA2 

#### Heuristics 4
- transaction hashes of EA1, EV and EA2 must be dissimilar
    - hA1 != hV != hA2

#### Heuristics 5
- transaction index of EA1 must be smaller than the transaction index of EV
- transaction index of EV must be smaller than the transaction index of EA2
    - iA1 < iV < i A2   

#### Heuristics 6
- the gas price of EA1 must be larger than the gas price of EV
- the gas price of EA2 must be less or equal to gas price of EV
    - gA1 > gV >= gA2 

### Implementation of Heuristics

In [1]:
from web3 import Web3


In [4]:
web3 = Web3(Web3.HTTPProvider("https://intensive-sly-mountain.quiknode.pro/a3f5256d7f2af6541d483cce3f1d49c94c01879e/"))
print("\033[92m"+str(web3.is_connected()))

True


In [3]:
BLOCK_NUMBER = 5574870
TRANSFER = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef" # ERC20 "Transfer"

events = web3.eth.filter({"fromBlock": BLOCK_NUMBER, "toBlock": BLOCK_NUMBER, "topics": [TRANSFER]}).get_all_entries()

coin_contract = events[0]["address"]
sender = events[0]["topics"][2].hex()
print(events[1])
print(coin_contract)
print(sender)

AttributeDict({'address': '0x86Fa049857E0209aa7D9e616F7eb3b3B78ECfdb0', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x000000000000000000000000010bdcf31074b87627683d83950b3183a10b813f'), HexBytes('0x00000000000000000000000003747f06215b44e498831da019b27f53e483599f')], 'data': HexBytes('0x00000000000000000000000000000000000000000000001e489f0784bcdc8400'), 'blockNumber': 5574870, 'transactionHash': HexBytes('0x1382efde35b60306e4667a8e8c5591175d17f6071a143d33785b8b6075191ba5'), 'transactionIndex': 14, 'blockHash': HexBytes('0x8afa98014df9eb5d6305d1e8caad29dd0c162d6c809009a75e1e148a118e63dd'), 'logIndex': 2, 'removed': False})
0xB62132e35a6c13ee1EE0f84dC5d40bad8d815206
0x000000000000000000000000b90812d219071878a75a59571c3279cfc3865ba5


In [6]:
transactions_by_address = {}

transactions = web3.eth.filter({"fromBlock": BLOCK_NUMBER, "toBlock": BLOCK_NUMBER, "topics": [TRANSFER]}).get_all_entries()

for transaction in transactions:
    
    token_contract_address = transaction["address"]
    
    if token_contract_address in transactions_by_address:
        transactions_by_address[token_contract_address].append(transaction)
    else:
        transactions_by_address[token_contract_address] = [transaction]


In [5]:
for token_contract_address in transactions_by_address:
    
    nr_of_transactions_with_same_coin = len(transactions_by_address[token_contract_address])
    
    # At least 3 transactions (A1, V, A2)
    if nr_of_transactions_with_same_coin <= 2:
        continue
    
    if token_contract_address != "0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB":
        continue
    
    print(token_contract_address)
    print(nr_of_transactions_with_same_coin)
    
    for transaction in transactions_by_address[token_contract_address]:
        
        address = web3.to_checksum_address(transaction["topics"][2].hex().replace("0x", "")[24:64])
        
        print(transaction["transactionIndex"])
        print(transaction["transactionHash"].hex())
        print(transaction["topics"])
        
        if address in ["0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C",
                       "0x8FB6840a46a8D143DaC1301F560976b953a095C5",
                       "0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C"]:
            pass
            print(transaction)
            

    
    print("\n")

0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB
8
94
0xedee87cdea91b70805184a1dbd32f689b02ff6f40579a601c2a4dd249d9b8090
[HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x000000000000000000000000238b7e54dfee4d8e98b8d1a78ab40dd94349bcfd'), HexBytes('0x000000000000000000000000cf1cc6ed5b653def7417e3fa93992c3ffe49139b')]
94
0xedee87cdea91b70805184a1dbd32f689b02ff6f40579a601c2a4dd249d9b8090
[HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x000000000000000000000000cf1cc6ed5b653def7417e3fa93992c3ffe49139b'), HexBytes('0x000000000000000000000000c9d9c248a71e5573a4f446b825f915c3e1359239')]
96
0xff17087b1cde666c6bd5022167a633ddd43b6ee2c929d1c98e493ffb63fe400a
[HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x000000000000000000000000238b7e54dfee4d8e98b8d1a78ab40dd94349bcfd'), HexBytes('0x000000000000000000000000cf1cc6ed5b653def7417e3fa93992c3ffe49139b')]
96
0xff17087b1cde666c6

In [8]:
BLOCK_NUMBER = 5574870
TRANSFER = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef" # ERC20 "Transfer"

for token_contract_address in transactions_by_address:
    
    nr_of_transactions_with_same_coin = len(transactions_by_address[token_contract_address])
    
    # At least 3 transactions (A1, V, A2)
    if nr_of_transactions_with_same_coin <= 2:
        continue
    
    if token_contract_address != "0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB":
        continue
    
    print(token_contract_address)
    print(nr_of_transactions_with_same_coin)
    print("\n")
    
    curr_transaction_index = None
    
    for transaction in transactions_by_address[token_contract_address]:
        
        if curr_transaction_index == transaction["transactionIndex"]:
            last_sender = transaction["topics"][1].hex().replace("0x", "")[24:64]
            last_receiver = transaction["topics"][2].hex().replace("0x", "")[24:64]
        else:
            first_sender = transaction["topics"][1].hex().replace("0x", "")[24:64]
            first_receiver = transaction["topics"][2].hex().replace("0x", "")[24:64]
            last_sender = transaction["topics"][1].hex().replace("0x", "")[24:64]
            last_receiver = transaction["topics"][2].hex().replace("0x", "")[24:64]
            
        curr_transaction_index = transaction["transactionIndex"]
        
        
        
        address = web3.to_checksum_address(transaction["topics"][2].hex().replace("0x", "")[24:64])
        
        print("Position in Block: ", transaction["transactionIndex"])
        #print("hash: ", transaction["transactionHash"].hex())
        print("Token address: ", transaction["address"])
        
        tx = web3.eth.get_transaction(transaction["transactionHash"].hex())
        
        print("Wallet: ", tx["from"]) 
        print("Sender: ", web3.to_checksum_address(transaction["topics"][1].hex().replace("0x", "")[24:64]))
        print("Receiver: ", web3.to_checksum_address(transaction["topics"][2].hex().replace("0x", "")[24:64]))

        #print(transaction["data"].hex())
        print("Amount: ", int(transaction["data"].hex().replace("0x", "")[0:64], 16) / 10 ** 18)
        print("Gas Price: ", tx["gasPrice"] / 10 ** 9)
        
        #print(tx["to"])
        
        if address in ["0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C",
                       "0x8FB6840a46a8D143DaC1301F560976b953a095C5",
                       "0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C"]:
        
            print("")
            
        print("\n")
    
    print("\n")

0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB
8


Position in Block:  94
Token address:  0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB
Wallet:  0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C
Sender:  0x238B7E54DfEE4d8e98b8D1A78AB40dd94349BcFd
Receiver:  0xcF1CC6eD5B653DeF7417E3fA93992c3FFe49139B
Amount:  2052115.257102965
Gas Price:  6.0


Position in Block:  94
Token address:  0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB
Wallet:  0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C
Sender:  0xcF1CC6eD5B653DeF7417E3fA93992c3FFe49139B
Receiver:  0xC9D9c248A71e5573A4f446B825F915C3e1359239
Amount:  2052115.257102965
Gas Price:  6.0


Position in Block:  96
Token address:  0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB
Wallet:  0xfF1b9745f68F84F036E5e92c920038d895FB701A
Sender:  0x238B7E54DfEE4d8e98b8D1A78AB40dd94349BcFd
Receiver:  0xcF1CC6eD5B653DeF7417E3fA93992c3FFe49139B
Amount:  2190274.9405929167
Gas Price:  5.98


Position in Block:  96
Token address:  0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB
Wallet:  0xfF1

### Approach

In [120]:
transactions = web3.eth.filter({"fromBlock": BLOCK_NUMBER, "toBlock": BLOCK_NUMBER, "topics": [TRANSFER]}).get_all_entries()

In [23]:
transaction_indexes = {}

for transaction in transactions:
    
    transaction_index = transaction["transactionIndex"]
    log_index = transaction["logIndex"]
    sender = web3.to_checksum_address(transaction["topics"][1].hex().replace("0x", "")[24:64])
    receiver = web3.to_checksum_address(transaction["topics"][2].hex().replace("0x", "")[24:64])
    address = transaction["address"]
    data =  int(transaction["data"].hex().replace("0x", "")[0:64], 16) 
    tx_hash = transaction["transactionHash"].hex()
    
    if transaction_index not in transaction_indexes:
        transaction_indexes[transaction_index] = {}
        transaction_indexes[transaction_index]["logIndexes"] = []
        transaction_indexes[transaction_index]["sender"] = []
        transaction_indexes[transaction_index]["receiver"] = []
        transaction_indexes[transaction_index]["address"] = []
        transaction_indexes[transaction_index]["data"] = []

    
    transaction_indexes[transaction_index]["logIndexes"].append(log_index)
    transaction_indexes[transaction_index]["sender"].append(sender)
    transaction_indexes[transaction_index]["receiver"].append(receiver)
    transaction_indexes[transaction_index]["address"].append(address)
    transaction_indexes[transaction_index]["data"].append(data)
    transaction_indexes[transaction_index]["transactionHash"] = tx_hash



    
print(transaction_indexes)

{13: {'logIndexes': [0], 'sender': ['0xabf3C296D4aA14052088948f81B67AC99CBa8241'], 'receiver': ['0xb90812d219071878A75a59571c3279CFc3865ba5'], 'address': ['0xB62132e35a6c13ee1EE0f84dC5d40bad8d815206'], 'data': [2962356792140000000000], 'transactionHash': '0xa2b78b6f4e95f568dc9bde036b1959cc53c62a4000e68f746467a308e0f102da'}, 14: {'logIndexes': [2], 'sender': ['0x010bDcf31074B87627683d83950B3183a10b813f'], 'receiver': ['0x03747F06215B44E498831dA019B27f53E483599F'], 'address': ['0x86Fa049857E0209aa7D9e616F7eb3b3B78ECfdb0'], 'data': [558635231770000000000], 'transactionHash': '0x1382efde35b60306e4667a8e8c5591175d17f6071a143d33785b8b6075191ba5'}, 15: {'logIndexes': [4], 'sender': ['0x2Ff2AD5ff29D51771E8B32229319AA4be9096255'], 'receiver': ['0x03747F06215B44E498831dA019B27f53E483599F'], 'address': ['0xf230b790E05390FC8295F4d3F60332c93BEd42e2'], 'data': [9783370106], 'transactionHash': '0xcfdf7e2afaaf5b67516c50ddba27f104420b85aec20b90f9059418afce06c431'}, 16: {'logIndexes': [6], 'sender': ['0

In [29]:
for tx_index in transaction_indexes:
    
    transaction = transaction_indexes[tx_index]
    
    tx_hash = transaction["transactionHash"]
    tx = web3.eth.get_transaction(tx_hash)
    
    print("Transaction index: ", tx_index)
    print("Token address: ", transaction["address"])
    print("Wallet: ", tx["from"]) 
    print("Gas Price: ", tx["gasPrice"] / 10 ** 9)
   
    first_log_idx = transaction["logIndexes"][0]
    last_log_idx = transaction["logIndexes"][-1]

    first_sender = transaction["sender"][0]
    first_receiver = transaction["receiver"][0]
    
    last_sender = transaction["sender"][-1]
    last_receiver = transaction["receiver"][-1]
    
    print(f"{first_log_idx}, {last_log_idx}")
    print(f"{first_sender}, {first_receiver}")
    print(f"{last_sender}, {last_receiver}")
    print("\n")
    

Transaction index:  13
Token address:  ['0xB62132e35a6c13ee1EE0f84dC5d40bad8d815206']
Wallet:  0xabf3C296D4aA14052088948f81B67AC99CBa8241
Gas Price:  55.0
0, 0
0xabf3C296D4aA14052088948f81B67AC99CBa8241, 0xb90812d219071878A75a59571c3279CFc3865ba5
0xabf3C296D4aA14052088948f81B67AC99CBa8241, 0xb90812d219071878A75a59571c3279CFc3865ba5


Transaction index:  14
Token address:  ['0x86Fa049857E0209aa7D9e616F7eb3b3B78ECfdb0']
Wallet:  0x03747F06215B44E498831dA019B27f53E483599F
Gas Price:  53.2
2, 2
0x010bDcf31074B87627683d83950B3183a10b813f, 0x03747F06215B44E498831dA019B27f53E483599F
0x010bDcf31074B87627683d83950B3183a10b813f, 0x03747F06215B44E498831dA019B27f53E483599F


Transaction index:  15
Token address:  ['0xf230b790E05390FC8295F4d3F60332c93BEd42e2']
Wallet:  0x03747F06215B44E498831dA019B27f53E483599F
Gas Price:  53.2
4, 4
0x2Ff2AD5ff29D51771E8B32229319AA4be9096255, 0x03747F06215B44E498831dA019B27f53E483599F
0x2Ff2AD5ff29D51771E8B32229319AA4be9096255, 0x03747F06215B44E498831dA019B27f53E48

In [137]:
def print_attribute_dict(attribute_dict):
    
    for key, value in attribute_dict.items():
        
        if key == "topics":
            for i, el in enumerate(value):
                value =  web3.to_checksum_address(el.hex().replace("0x", "")[24:64])
                print(f"Topic {i}: {value}")
            continue
        
        try:
            value = value.hex()
        except:
            pass
        print(f"{key}: {value}")

In [185]:
for t in transactions:
    
    if t["transactionIndex"] not in [94]:
        continue
    print_attribute_dict(t)
    print("\n")

address: 0xc0829421C1d260BD3cB3E0F06cfE2D52db2cE315
Topic 0: 0xFC378dAA952ba7f163c4a11628f55a4df523b3EF
Topic 1: 0xc0829421C1d260BD3cB3E0F06cfE2D52db2cE315
Topic 2: 0xcF1CC6eD5B653DeF7417E3fA93992c3FFe49139B
data: 0x00000000000000000000000000000000000000000000000055c6d2da9d849600
blockNumber: 5574870
transactionHash: 0xedee87cdea91b70805184a1dbd32f689b02ff6f40579a601c2a4dd249d9b8090
transactionIndex: 94
blockHash: 0x8afa98014df9eb5d6305d1e8caad29dd0c162d6c809009a75e1e148a118e63dd
logIndex: 63
removed: False


address: 0xc0829421C1d260BD3cB3E0F06cfE2D52db2cE315
Topic 0: 0xFC378dAA952ba7f163c4a11628f55a4df523b3EF
Topic 1: 0xcF1CC6eD5B653DeF7417E3fA93992c3FFe49139B
Topic 2: 0xc6725aE749677f21E4d8f85F41cFB6DE49b9Db29
data: 0x00000000000000000000000000000000000000000000000055c6d2da9d849600
blockNumber: 5574870
transactionHash: 0xedee87cdea91b70805184a1dbd32f689b02ff6f40579a601c2a4dd249d9b8090
transactionIndex: 94
blockHash: 0x8afa98014df9eb5d6305d1e8caad29dd0c162d6c809009a75e1e148a118e63dd


In [84]:
transactions = web3.eth.filter({"fromBlock": BLOCK_NUMBER, "toBlock": BLOCK_NUMBER, "topics": [TRANSFER]}).get_all_entries()


In [76]:
for transaction in transactions:
    
    address = web3.to_checksum_address(transaction["topics"][2].hex().replace("0x", "")[24:64])
    
    if (address in 
            ["0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C",
             "0x8FB6840a46a8D143DaC1301F560976b953a095C5",
             "0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C"]):
        
        print(transaction)

AttributeDict({'address': '0x9a0242b7a33DAcbe40eDb927834F96eB39f8fBCB', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x000000000000000000000000cf1cc6ed5b653def7417e3fa93992c3ffe49139b'), HexBytes('0x0000000000000000000000008fb6840a46a8d143dac1301f560976b953a095c5')], 'data': HexBytes('0x00000000000000000000000000000000000000000001494c2ecef47c9fae5795'), 'blockNumber': 5574870, 'transactionHash': HexBytes('0xb46caa3c254f3e2050818a1ce3ea6b01f96011b403253c620c9a1720c4a6612a'), 'transactionIndex': 98, 'blockHash': HexBytes('0x8afa98014df9eb5d6305d1e8caad29dd0c162d6c809009a75e1e148a118e63dd'), 'logIndex': 115, 'removed': False})


In [100]:
block = web3.eth.get_block(BLOCK_NUMBER, full_transactions=True)

for transaction in block.transactions:
        
    address = transaction["from"]
    
    if (address in 
            ["0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C",
             "0x8FB6840a46a8D143DaC1301F560976b953a095C5",
             "0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C"]):
        
        print(transaction)
        print(transaction["from"])
        print(transaction["to"])
        print(transaction["transactionIndex"])

AttributeDict({'blockHash': HexBytes('0x8afa98014df9eb5d6305d1e8caad29dd0c162d6c809009a75e1e148a118e63dd'), 'blockNumber': 5574870, 'from': '0x4fCc2FF6c75923D33B4F5aF4C524461014B2EE1C', 'gas': 840000, 'gasPrice': 6000000000, 'hash': HexBytes('0xedee87cdea91b70805184a1dbd32f689b02ff6f40579a601c2a4dd249d9b8090'), 'input': HexBytes('0x0000012e47dfea9ef1598627bea38283340dc8a2858c098600000000000000000000000000000000000000000000000013d008dceff6b7d1eacb9cd9d5644b711adef892000000000000000055c6d2da9d8496000000000000001955926931443fc21f61000000000000000000000000000000000000000000000000000000005510e41b0fe39619b3edf162c3231beda7a7a28e598d1ba341ed47df24d021b680e3bf9c36a99c5b9671cbbe840c9619fde7caceb7c849dbe87c1e916d30052e71175cb100000000000000000000000000000000000000000000000000000000000000030000000000000000000000000000000000000000000000000000000000000000'), 'nonce': 1819, 'to': '0xC9D9c248A71e5573A4f446B825F915C3e1359239', 'transactionIndex': 94, 'value': 0, 'type': 0, 'chainId': 1, 'v': 38, 'r': 